In [191]:
import pandas as pd
import numpy as np
import datetime
import os

# Read in Data

In [192]:
cwd = '../data/raw_data/'

t = pd.read_csv('../data/california_tweets.csv') #read in data
mobility = pd.read_csv(cwd+'2020_US_Region_Mobility_Report.csv') #read in data
mobility.head()
max(mobility['date'])

'2020-12-01'

# Mobility Growth Rate

In [193]:
mobility['date'] = pd.to_datetime(mobility['date']) #convert str object to datetime
date_after = pd.to_datetime('2020-03-20') #this is where our twitter data begins
date_before = pd.to_datetime('2020-12-01') #this is where our twitter data ends
mobility = mobility[mobility['date'] >= date_after]
mobility = mobility[mobility['date'] <= date_before]
mobility = mobility.rename(columns={"retail_and_recreation_percent_change_from_baseline": "retail_rec", 
                                    "grocery_and_pharmacy_percent_change_from_baseline": "groc_pharm",
                                   "parks_percent_change_from_baseline": "parks",
                                   "transit_stations_percent_change_from_baseline": "transit",
                                   "workplaces_percent_change_from_baseline": "workplaces",
                                   "residential_percent_change_from_baseline": "residential"})
ca_mobility = mobility[mobility.sub_region_1 == 'California'] #get just mobility for california
ca_mobility.head(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_rec,groc_pharm,parks,transit,workplaces,residential
43409,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-20,-40.0,-3.0,-22.0,-46.0,-44.0,21.0
43410,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-21,-48.0,-13.0,-22.0,-47.0,-33.0,16.0
43411,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-22,-50.0,-22.0,-24.0,-52.0,-36.0,14.0
43412,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-23,-41.0,-18.0,-28.0,-52.0,-49.0,21.0
43413,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-24,-42.0,-18.0,-31.0,-52.0,-50.0,22.0


In [211]:
ca = ca_mobility.drop(columns = ['census_fips_code','iso_3166_2_code', 'metro_area', 'country_region_code', 'country_region', 'sub_region_1'])


ca['work_DOD'] = ca_mobility.workplaces.pct_change().mul(100).round(2)
ca['res_DOD'] = ca_mobility.residential.pct_change().mul(100).round(2)

ca

,county,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,workplace_DOD
43409,NaN,2020-03-20,-40.0,-3.0,-22.0,-46.0,-44.0,21.0,NaN
43410,NaN,2020-03-21,-48.0,-13.0,-22.0,-47.0,-33.0,16.0,-25.00
43411,NaN,2020-03-22,-50.0,-22.0,-24.0,-52.0,-36.0,14.0,9.09
43412,NaN,2020-03-23,-41.0,-18.0,-28.0,-52.0,-49.0,21.0,36.11
43413,NaN,2020-03-24,-42.0,-18.0,-31.0,-52.0,-50.0,22.0,2.04
...,...,...,...,...,...,...,...,...,...
59466,Yuba County,2020-11-27,-26.0,-18.0,35.0,-46.0,-61.0,17.0,-19.74
59467,Yuba County,2020-11-28,-19.0,-5.0,37.0,-40.0,-17.0,7.0,-72.13
59468,Yuba County,2020-11-29,-6.0,-3.0,36.0,NaN,-17.0,4.0,0.00
59469,Yuba County,2020-11-30,-17.0,2.0,NaN,-9.0,-30.0,8.0,76.47


# Clean up Tweets

In [194]:
t.head(5)

,Unnamed: 0,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang,longitude,latitude,state,county
0,177,2020-03-20,"My office, staying at home, helping Stop the C...","[33.6638, -118.0076]","{'id': '80eb17ffe368fc9a', 'url': 'https://api...",False,"[0, 132]","{'id': 29916709, 'id_str': '29916709', 'name':...",1240790800088403968,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-118.0076,33.6638,California,Orange County
1,180,2020-03-20,Corona CA Thu Mar 19th PM Forecast: TONIGHT Ch...,"[33.8752, -117.5655]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 94]","{'id': 354691403, 'id_str': '354691403', 'name...",1240791066011488257,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,en,-117.5655,33.8752,California,NaN
2,200,2020-03-20,“COLDER” should i shoot this video next??? @ C...,"[33.8753, -117.566]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 87]","{'id': 867752113794818048, 'id_str': '86775211...",1240796072735375366,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.5660,33.8753,California,NaN
3,206,2020-03-20,.\n👆🏾ClickFollow &amp;/or a “😂”👆🏾 \n#DM to 5 p...,"[34.0522, -118.243]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 235]","{'id': 23940847, 'id_str': '23940847', 'name':...",1240799104009150464,"{'hashtags': [{'text': 'DM', 'indices': [34, 3...",False,en,-118.2430,34.0522,California,Los Angeles County
4,216,2020-03-20,I’ll be on Hotepish tonight with my boy @dwann...,"[34.0522, -118.243]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 239]","{'id': 573197933, 'id_str': '573197933', 'name...",1240804304463822849,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-118.2430,34.0522,California,Los Angeles County


In [195]:
missing_county = t[t['county'].isna()]
len(missing_county)

376

In [196]:
#riverside bounds
lat = (33.447181, 34.078074)
long = (-117.674869, -114.517813)

long_boolean = missing_county['longitude'].between(long[0], long[1],  inclusive=True)
lat_boolean = missing_county['latitude'].between(lat[0], lat[1],  inclusive=True)
riverside_guess = missing_county[long_boolean & lat_boolean]

In [197]:
len(missing_county) == len(riverside_guess)

True

Since the missing county coordinates all fall within Riverside county, let's plug in for the NaNs

In [198]:
t.county = t.county.fillna('Riverside County')
t.county = t.county.replace('Riverside', 'Riverside County')

We'll have to perform a merge that matches up both the date and county for the dataframe t. 


In [199]:
t.created_at = pd.to_datetime(t.created_at) #convert the data column to timestamp for merge to compare
t = t.rename(columns = {"created_at": "date"}) #rename for cols to match
ca_mobility = ca_mobility.rename(columns = {"sub_region_2": "county"}) #rename for cols to match

# Merge Tweets and Mobility

In [200]:
tweet_mobility = t.merge(ca_mobility, how = 'left', on = ['county', 'date'])

In [207]:
tweet_mobility.head(2)

,Unnamed: 0,date,full_text,geo,place,truncated,display_text_range,user,id,entities,...,sub_region_1,metro_area,iso_3166_2_code,census_fips_code,retail_rec,groc_pharm,parks,transit,workplaces,residential
0,177,2020-03-20,"My office, staying at home, helping Stop the C...","[33.6638, -118.0076]","{'id': '80eb17ffe368fc9a', 'url': 'https://api...",False,"[0, 132]","{'id': 29916709, 'id_str': '29916709', 'name':...",1240790800088403968,"{'hashtags': [], 'symbols': [], 'user_mentions...",...,California,NaN,NaN,6059.0,-42.0,-3.0,-22.0,-46.0,-44.0,24.0
1,180,2020-03-20,Corona CA Thu Mar 19th PM Forecast: TONIGHT Ch...,"[33.8752, -117.5655]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 94]","{'id': 354691403, 'id_str': '354691403', 'name...",1240791066011488257,"{'hashtags': [], 'symbols': [], 'user_mentions...",...,California,NaN,NaN,6065.0,-33.0,4.0,-35.0,-30.0,-38.0,18.0


In [202]:
len(ca_tweets) == len(tweet_mobility) #sanity_check

True

In [203]:
tw = tweet_mobility.drop(columns = ['Unnamed: 0', 'census_fips_code','iso_3166_2_code', 'metro_area', 'place', 'lang', 'country_region_code', 'country_region', 'sub_region_1'])

In [204]:
tw.set_index('id')

,date,full_text,geo,truncated,display_text_range,user,entities,possibly_sensitive,longitude,latitude,state,county,retail_rec,groc_pharm,parks,transit,workplaces,residential
id,,,,,,,,,,,,,,,,,,
1240790800088403968,2020-03-20,"My office, staying at home, helping Stop the C...","[33.6638, -118.0076]",False,"[0, 132]","{'id': 29916709, 'id_str': '29916709', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.007600,33.663800,California,Orange County,-42.0,-3.0,-22.0,-46.0,-44.0,24.0
1240791066011488257,2020-03-20,Corona CA Thu Mar 19th PM Forecast: TONIGHT Ch...,"[33.8752, -117.5655]",False,"[0, 94]","{'id': 354691403, 'id_str': '354691403', 'name...","{'hashtags': [], 'symbols': [], 'user_mentions...",True,-117.565500,33.875200,California,Riverside County,-33.0,4.0,-35.0,-30.0,-38.0,18.0
1240796072735375366,2020-03-20,“COLDER” should i shoot this video next??? @ C...,"[33.8753, -117.566]",False,"[0, 87]","{'id': 867752113794818048, 'id_str': '86775211...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-117.566000,33.875300,California,Riverside County,-33.0,4.0,-35.0,-30.0,-38.0,18.0
1240799104009150464,2020-03-20,.\n👆🏾ClickFollow &amp;/or a “😂”👆🏾 \n#DM to 5 p...,"[34.0522, -118.243]",False,"[0, 235]","{'id': 23940847, 'id_str': '23940847', 'name':...","{'hashtags': [{'text': 'DM', 'indices': [34, 3...",False,-118.243000,34.052200,California,Los Angeles County,-40.0,-5.0,-29.0,-44.0,-41.0,22.0
1240804304463822849,2020-03-20,I’ll be on Hotepish tonight with my boy @dwann...,"[34.0522, -118.243]",False,"[0, 239]","{'id': 573197933, 'id_str': '573197933', 'name...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.243000,34.052200,California,Los Angeles County,-40.0,-5.0,-29.0,-44.0,-41.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252381368119525383,2020-04-20,A little ray of sunshine on these dark days #c...,"[33.88366, -117.5646]",False,"[0, 202]","{'id': 74402020, 'id_str': '74402020', 'name':...","{'hashtags': [{'text': 'cheflife', 'indices': ...",False,-117.564600,33.883660,California,Riverside County,-52.0,-15.0,-48.0,-40.0,-44.0,18.0
1252382207793983495,2020-04-20,#snapchat👻 addiction #covid #quarantinelife #s...,"[34.17541248, -118.33602388]",False,"[0, 96]","{'id': 156361436, 'id_str': '156361436', 'name...","{'hashtags': [{'text': 'snapchat', 'indices': ...",False,-118.336024,34.175412,California,Los Angeles County,-54.0,-22.0,-38.0,-50.0,-49.0,22.0
1252382248411660288,2020-04-20,"🦋 On a #beautiful, #sunny #spring day, #strang...","[38.5763, -121.4927]",False,"[0, 201]","{'id': 114578117, 'id_str': '114578117', 'name...","{'hashtags': [{'text': 'beautiful', 'indices':...",False,-121.492700,38.576300,California,Sacramento County,-49.0,-15.0,-6.0,-55.0,-49.0,19.0


In [205]:
tw.to_csv('../data/tweet_mobility.csv')